In [ ]:
# Required Libraries
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

# Parameters
num_classes = 10
img_rows, img_cols = 28, 28

# Load Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess Data
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# Normalize Data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert Class Vectors to Binary Class Matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

# Learning Rate Scheduler
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train Model
model.fit(x_train, y_train, batch_size=64, epochs=30,
          validation_data=(x_test, y_test), callbacks=[annealer])

# Save Model
model.save('mnist_cnn_model.h5')


11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/30
938/938 [==============================] - 29s 12ms/step - loss: 0.2023 - accuracy: 0.9388 - val_loss: 0.0514 - val_accuracy: 0.9827 - lr: 0.0010
Epoch 2/30
938/938 [==============================] - 11s 11ms/step - loss: 0.0680 - accuracy: 0.9792 - val_loss: 0.0292 - val_accuracy: 0.9905 - lr: 9.5000e-04
Epoch 3/30
938/938 [==============================] - 11s 11ms/step - loss: 0.0491 - accuracy: 0.9847 - val_loss: 0.0235 - val_accuracy: 0.9924 - lr: 9.0250e-04
Epoch 4/30
938/938 [==============================] - 11s 12ms/step - loss: 0.0405 - accuracy: 0.9877 - val_loss: 0.0211 - val_accuracy: 0.9929 - lr: 8.5737e-04
Epoch 5/30
938/938 [==============================] - 11s 12ms/step - loss: 0.0370 - accuracy: 0.9885 - val_loss: 0.0302 - val_accuracy: 0.9900 - lr: 8.1451e-04
Epoch 6/30
938/938 [==============================] - 11s 12ms/step - loss: 0.0329 - accuracy: 0.9902 - val_loss: 0.0229 - val_accurac

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers.legacy import SGD

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape((X_train.shape[0], num_pixels)).astype('float32') / 255
X_test = X_test.reshape((X_test.shape[0], num_pixels)).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create an MLP model
model = Sequential()
model.add(Dense(1024, input_dim=num_pixels, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

# use SGD optimizer
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Save the model at every epoch in which the validation accuracy improves
checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

# Reduce learning rate when the validation loss has stopped decreasing
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=128, verbose=2,
          callbacks=[checkpoint, reduce_lr])


11490434/11490434 [==============================] - 2s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
469/469 - 6s - loss: 0.5467 - accuracy: 0.8281 - val_loss: 0.1740 - val_accuracy: 0.9476 - lr: 0.0100 - 6s/epoch - 12ms/step
Epoch 2/100
469/469 - 2s - loss: 0.1969 - accuracy: 0.9416 - val_loss: 0.1214 - val_accuracy: 0.9620 - lr: 0.0100 - 2s/epoch - 3ms/step
Epoch 3/100
469/469 - 2s - loss: 0.1398 - accuracy: 0.9578 - val_loss: 0.0964 - val_accuracy: 0.9691 - lr: 0.0100 - 2s/epoch - 5ms/step
Epoch 4/100
469/469 - 2s - loss: 0.1065 - accuracy: 0.9678 - val_loss: 0.0850 - val_accuracy: 0.9736 - lr: 0.0100 - 2s/epoch - 4ms/step
Epoch 5/100
469/469 - 1s - loss: 0.0888 - accuracy: 0.9736 - val_loss: 0.0728 - val_accuracy: 0.9768 - lr: 0.0100 - 1s/epoch - 3ms/step
Epoch 6/100
469/469 - 2s - loss: 0.0747 - accuracy: 0.9776 - val_loss: 0.0728 - val_accuracy: 0.9772 - lr: 0.0100 - 2s/epoch - 3ms/step
Epoch 7/100
469/469 - 2s - loss: 0.0631 - accuracy: 0.9801 - val_loss: 0.0608 - val_accuracy: 0.9820 - lr: 0.0100 - 2s/epoch - 4ms/step
Epoch 8/100
469/469 - 2s - loss: 0.0553 - accur